# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [27]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_data = pd.read_csv("../WeatherPy/weather_data.csv")
weather_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lnt,Max Temp,Wind Speed
0,Atuona,100,PF,1579320342,73,-9.80,-139.03,81.70,17.20
1,Ushuaia,90,AR,1579320342,81,-54.80,-68.30,46.40,12.75
2,Ribeira Grande,60,PT,1579320342,92,38.52,-28.70,58.84,6.87
3,Punta Arenas,40,CL,1579320342,76,-53.15,-70.92,50.00,19.46
4,Quelimane,40,MZ,1579320342,94,-17.88,36.89,80.60,3.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lnt"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,  dissipating=False, max_intensity=10, point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [42]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness less than 20
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] < 20) & \
                                   (weather_data["Max Temp"] > 50) & (weather_data["Max Temp"] < 90)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lnt,Max Temp,Wind Speed
28,Olavarria,0,AR,1579320346,74,-36.89,-60.32,66.00,1.01
72,Los Llanos de Aridane,0,ES,1579320353,67,28.66,-17.92,60.80,9.17
114,Arrecife,0,ES,1579320359,77,28.96,-13.55,62.60,9.17
115,Ancud,0,CL,1579320359,87,-41.87,-73.82,55.40,9.17
118,Mathbaria,0,BD,1579320082,52,22.29,89.96,73.72,4.85
120,Lebu,0,CL,1579320360,80,-37.62,-73.65,59.11,4.65
132,Le Vauclin,4,MQ,1579320362,94,14.55,-60.84,73.40,4.70
136,Sur,0,OM,1579320362,58,22.57,59.53,62.42,8.25
138,Lagoa,0,PT,1579320363,86,39.05,-27.98,61.92,7.90
159,Ghanzi,0,BW,1579320366,73,-21.57,21.78,66.63,3.89


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
hotel_df = narrowed_city_df[["City", "Lat", "Lnt"]].reset_index()
hotel_df["Hotel Name"] = ""

hotel_df

,index,City,Lat,Lnt,Hotel Name
0,28,Olavarria,-36.89,-60.32,
1,72,Los Llanos de Aridane,28.66,-17.92,
2,114,Arrecife,28.96,-13.55,
3,115,Ancud,-41.87,-73.82,
4,118,Mathbaria,22.29,89.96,
5,120,Lebu,-37.62,-73.65,
6,132,Le Vauclin,14.55,-60.84,
7,136,Sur,22.57,59.53,
8,138,Lagoa,39.05,-27.98,
9,159,Ghanzi,-21.57,21.78,


In [45]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [46]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lnt = row["Lnt"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lnt}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: Olavarria.
Closest hotel in Olavarria is Hotel Lleras Park Boutique.
------------
Retrieving Results for Index 1: Los Llanos de Aridane.
Closest hotel in Los Llanos de Aridane is Hotel Hacienda de Abajo.
------------
Retrieving Results for Index 2: Arrecife.
Closest hotel in Arrecife is Arrecife Gran Hotel & Spa.
------------
Retrieving Results for Index 3: Ancud.
Closest hotel in Ancud is Panamericana Hotel Ancud.
------------
Retrieving Results for Index 4: Mathbaria.
Closest hotel in Mathbaria is প্রিন্স হোটেল মঠবাড়িয়া.
------------
Retrieving Results for Index 5: Lebu.
Closest hotel in Lebu is Hotel Plaza Lebu.
------------
Retrieving Results for Index 6: Le Vauclin.
Closest hotel in Le Vauclin is Le Cap Est Lagoon Resort & Spa.
------------
Retrieving Results for Index 7: Sur.
Closest hotel in Sur is Sur Plaza Hotel.
------------
Retrieving Results for Index 8: Lagoa.
Closest hotel in Lagoa is Graciosa Resort - Biosphere Island Hotel.
------------
R

In [47]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer ontop of heat map


# Display Map